In [1]:
import asyncio
import requests
import re
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
apiKey = '07cf7672adc37b6bbf593528a743f1e1'
apiSecret = 'C/LMXFbFlxPIh4324lEVUjD5Ei9YmSJDJKe24CWtLT4apA6BsVBL/RcnbDclbynTBls5iIs4PHinOsocWFALNg=='
passPhrase = 'dx6f9xhn9ue'

In [18]:
def cbpGetHistoricRates(market='LINK-USD', granularity=86400, iso8601start='', iso8601end=''):
    
    p = re.compile(r"^[A-Z]{3,4}\-[A-Z]{3,4}$")
    if not p.match(market):
        raise TypeError('Coinabse Pro market required.')
        
    if not isinstance(granularity, int):
        raise TypeError('Granularity integer required.')
        
    if not granularity in [60, 300, 900, 3600, 21600, 86400]:
        raise TypeError('Granularity options: 60, 300, 900, 3600, 21600, 86400')
        
    if not isinstance(iso8601start, str):
        raise TypeError('ISO8601 start integer is string required')
        
    if not isinstance(iso8601end, str):
        raise TypeError('ISO8601 end integer is string required')
        
    api = 'https://api.pro.coinbase.com/products/' + market + '/candles?granularity=' + str(granularity) + '&start=' + iso8601start + '&end=' + iso8601end
    resp = requests.get(api, verify=False, timeout=5)
    if resp.status_code != 200:
        raise Exception('Get' + api + '{}' .format(resp.status_code))
        
    df = pd.DataFrame(resp.json(), columns = ['iso8601','low','high','open','close','volume'])
    df = df.iloc[::-1].reset_index()
    
    for index, row in df.iterrows():
        iso8601 = datetime.fromtimestamp(row['iso8601'])
        df.at[index, 'datetime'] = datetime.strftime(iso8601, "%d/%m/%Y %H:%M:%S")
        
    df = df.reindex(columns = ['iso8601','datetime','low','high','open','close','volume'])
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    return df

In [19]:
data = cbpGetHistoricRates()

C:\Users\jwkon\anaconda3\envs\API\lib\site-packages\requests\packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


In [20]:
data

,iso8601,datetime,low,high,open,close,volume
0,1583884800,2020-10-03 20:00:00,3.55000,4.11249,4.07642,3.82519,4459905.80
1,1583971200,2020-11-03 20:00:00,1.86000,3.85809,3.82550,2.13985,23787442.87
2,1584057600,2020-12-03 20:00:00,1.51500,2.88880,2.12010,2.46196,24820702.08
3,1584144000,2020-03-13 20:00:00,2.07000,2.52030,2.45391,2.14352,6516263.86
4,1584230400,2020-03-14 20:00:00,2.02000,2.47000,2.13547,2.08094,8844392.69
...,...,...,...,...,...,...,...
295,1609372800,2020-12-30 19:00:00,10.94620,11.76268,11.25501,11.25489,2653818.57
296,1609459200,2020-12-31 19:00:00,11.08824,12.47666,11.24796,11.88126,3238494.13
297,1609545600,2021-01-01 19:00:00,11.46666,12.45457,11.88126,12.23513,3960880.09
298,1609632000,2021-02-01 19:00:00,11.83000,14.03648,12.23427,13.70702,6836216.01


In [21]:
data = data.drop(['iso8601'], axis=1)
data['date'] = pd.to_datetime(data['datetime']).dt.date
#data = data.set_index(pd.DatetimeIndex(data['date'].values))
data = data.drop(columns=['datetime'])
data.head()

,low,high,open,close,volume,date
0,3.550,4.11249,4.07642,3.82519,4459905.80,2020-10-03
1,1.860,3.85809,3.82550,2.13985,23787442.87,2020-11-03
2,1.515,2.88880,2.12010,2.46196,24820702.08,2020-12-03
3,2.070,2.52030,2.45391,2.14352,6516263.86,2020-03-13
4,2.020,2.47000,2.13547,2.08094,8844392.69,2020-03-14


In [22]:
data.to_csv('HistoricRates_LINK', index=False)